In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import re

# --- Krok 1: Wczytanie i przygotowanie danych ---
input_filename = 'saleflats_mazowieckie_c.csv'
print(f"Wczytywanie pliku: {input_filename}...")
try:
    df_warszawa = pd.read_csv(input_filename, sep=',', header=None, on_bad_lines='skip', low_memory=False)
    print("Plik wczytany pomyślnie.")
except FileNotFoundError:
    print(f"BŁĄD: Plik '{input_filename}' nie został znaleziony.")
    raise

# --- Krok 2: Nadanie nazw kluczowym kolumnom ---
# Nadajemy nazwy tylko tym kolumnom, których będziemy używać
df_warszawa.rename(columns={2: 'Location', 59: 'UlicaID'}, inplace=True)
print("Nazwy kluczowych kolumn nadane.")

# --- Krok 3: Budowa słownika ID -> Nazwa (NOWA, UPROSZCZONA METODA) ---
print("Budowanie słownika na podstawie ID ulicy i tekstu lokalizacji...")

# Upewniamy się, że dane są w odpowiednich typach
df_warszawa['UlicaID'] = pd.to_numeric(df_warszawa['UlicaID'], errors='coerce').fillna(0).astype(int)
df_warszawa['Location'] = df_warszawa['Location'].astype(str)

# Usuwamy wiersze bez ID ulicy
df_streets = df_warszawa[df_warszawa['UlicaID'] != 0].copy()

# Funkcja do wyciągania ostatniego elementu z lokalizacji
def get_last_location_part(location_str):
    if not isinstance(location_str, str) or len(location_str.strip()) == 0:
        return np.nan
    # Bierzemy ostatni element po przecinku i czyścimy go
    last_part = location_str.split(',')[-1].strip().lower()
    # Usuwamy typowe przedrostki
    last_part = re.sub(r'^(ul\.|al\.|pl\.)\s*', '', last_part)
    return last_part

df_streets['Ulica_Nazwa'] = df_streets['Location'].apply(get_last_location_part)

# Usuwamy wiersze, gdzie nie udało się wyciągnąć nazwy
df_streets.dropna(subset=['Ulica_Nazwa'], inplace=True)

# Tworzymy słownik, biorąc pierwszą znalezioną nazwę dla danego ID
street_dictionary = df_streets.drop_duplicates(subset='UlicaID').set_index('UlicaID')['Ulica_Nazwa'].to_dict()

# --- Krok 4: Zapis i wyświetlenie słownika ---
print(f"Zbudowano słownik zawierający {len(street_dictionary)} unikalnych wpisów.")

if street_dictionary:
    slownik_df = pd.DataFrame(list(street_dictionary.items()), columns=['UlicaID', 'Nazwa_Ulicy'])
    
    output_filename = 'slownik_ulic_warszawa.csv'
    print(f"Zapisywanie słownika do pliku: {output_filename}...")
    slownik_df.to_csv(output_filename, index=False, sep=';')
    print("Zapis zakończony sukcesem!")

    print("\nPróbka 20 losowych wierszy ze słownika:")
    display(slownik_df.sample(20, random_state=42))
else:
    print("\nNiestety, nie udało się zbudować słownika. Sprawdź strukturę pliku wejściowego.")

Wczytywanie pliku: saleflats_mazowieckie_c.csv...
Plik wczytany pomyślnie.
Nazwy kluczowych kolumn nadane.
Budowanie słownika na podstawie ID ulicy i tekstu lokalizacji...
Zbudowano słownik zawierający 2708 unikalnych wpisów.
Zapisywanie słownika do pliku: slownik_ulic_warszawa.csv...
Zapis zakończony sukcesem!

Próbka 20 losowych wierszy ze słownika:


,UlicaID,Nazwa_Ulicy
1044,515548,14
439,361128,15
1729,514352,14
296,414096,2
2211,480945,18
479,364478,14
1084,376107,10
2080,500370,2
2204,305815,4
318,295727,10


In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
import re

# --- Krok 1: Wczytanie danych ---
input_filename = 'saleflats_mazowieckie_c.csv'
print(f"Wczytywanie pliku: {input_filename}...")
try:
    df_warszawa = pd.read_csv(input_filename, sep=',', header=None, on_bad_lines='skip', low_memory=False)
    print("Plik wczytany pomyślnie.")
except FileNotFoundError:
    print(f"BŁĄD: Plik '{input_filename}' nie został znaleziony.")
    raise

# --- Krok 2: Nadanie nazw kluczowym kolumnom ---
# ZMIANA: Nadajemy nazwy tylko tym kolumnom, których naprawdę potrzebujemy
# Kolumna 43 -> LocationText
# Kolumna 59 -> UlicaID
df_warszawa.rename(columns={43: 'LocationText', 59: 'UlicaID'}, inplace=True)
print("Nazwy kluczowych kolumn nadane.")

# --- Krok 3: Budowa słownika ID -> Nazwa ---
print("Budowanie słownika na podstawie ID ulicy i tekstu lokalizacji...")

# Upewniamy się, że dane są w odpowiednich typach
df_warszawa['UlicaID'] = pd.to_numeric(df_warszawa['UlicaID'], errors='coerce').fillna(0).astype(int)
df_warszawa['LocationText'] = df_warszawa['LocationText'].astype(str)

# Usuwamy wiersze bez ID ulicy
df_streets = df_warszawa[df_warszawa['UlicaID'] != 0].copy()

# Funkcja do wyciągania ostatniego elementu z lokalizacji
def get_last_location_part(location_str):
    if not isinstance(location_str, str) or len(location_str.strip()) == 0:
        return np.nan
    # Bierzemy ostatni element po przecinku i czyścimy go
    last_part = location_str.split(',')[-1].strip().lower()
    # Usuwamy typowe przedrostki
    last_part = re.sub(r'^(ul\.|al\.|pl\.)\s*', '', last_part)
    return last_part

df_streets['Ulica_Nazwa'] = df_streets['LocationText'].apply(get_last_location_part)

# Usuwamy wiersze, gdzie nie udało się wyciągnąć nazwy
df_streets.dropna(subset=['Ulica_Nazwa'], inplace=True)
df_streets = df_streets[df_streets['Ulica_Nazwa'] != 'nan'] # Dodatkowe czyszczenie

# Tworzymy słownik, biorąc pierwszą znalezioną nazwę dla danego ID
street_dictionary = df_streets.drop_duplicates(subset='UlicaID').set_index('UlicaID')['Ulica_Nazwa'].to_dict()

# --- Krok 4: Zapis i wyświetlenie słownika ---
print(f"Zbudowano słownik zawierający {len(street_dictionary)} unikalnych wpisów.")

if street_dictionary:
    slownik_df = pd.DataFrame(list(street_dictionary.items()), columns=['UlicaID', 'Nazwa_Ulicy'])
    
    output_filename = 'slownik_ulic_warszawa.csv'
    print(f"Zapisywanie słownika do pliku: {output_filename}...")
    slownik_df.to_csv(output_filename, index=False, sep=';')
    print("Zapis zakończony sukcesem!")

    print("\nPróbka 20 losowych wierszy ze słownika:")
    display(slownik_df.sample(20, random_state=42))
else:
    print("\nNiestety, nie udało się zbudować słownika. Sprawdź strukturę pliku wejściowego.")

Wczytywanie pliku: saleflats_mazowieckie_c.csv...
Plik wczytany pomyślnie.
Nazwy kluczowych kolumn nadane.
Budowanie słownika na podstawie ID ulicy i tekstu lokalizacji...
Zbudowano słownik zawierający 2708 unikalnych wpisów.
Zapisywanie słownika do pliku: slownik_ulic_warszawa.csv...
Zapis zakończony sukcesem!

Próbka 20 losowych wierszy ze słownika:


,UlicaID,Nazwa_Ulicy
1044,515548,"kompanii ak goplana"""
439,361128,szegedyńska
1729,514352,pielęgniarek
296,414096,kwitnąca
2211,480945,tawerny
479,364478,belgijska
1084,376107,lutocińska
2080,500370,cicha dolina
2204,305815,leśniczówka
318,295727,dymińska


In [4]:
import pandas as pd
import numpy as np
from IPython.display import display

# === SEKCJA 2: BUDOWA SŁOWNIKA Z PLIKU _c.csv (POPRAWIONA) ===

print("--- Krok 1: Wczytanie danych ---")
try:
    # Plik z danymi dla Warszawy
    df_warszawa = pd.read_csv('saleflats_mazowieckie_c.csv', sep=',', header=None, on_bad_lines='skip', low_memory=False)
    print("Plik 'saleflats_mazowieckie_c.csv' wczytany.")
    
    # Oficjalna baza lokalizacji do weryfikacji
    df_meta_raw = pd.read_csv('lokalizacja.csv', sep=',', header=None, on_bad_lines='skip')
    print("Plik 'lokalizacja.csv' wczytany.")
except FileNotFoundError as e:
    print(f"BŁĄD: Nie znaleziono pliku: {e.filename}")
    raise

# --- Krok 2: Nadanie nazw kolumnom i budowa słownika wydedukowanego ---
# Używamy tylko kolumn 43 (LocationText) i 59 (UlicaID)
df_warszawa.rename(columns={43: 'LocationText', 59: 'UlicaID'}, inplace=True)
df_warszawa['UlicaID'] = pd.to_numeric(df_warszawa['UlicaID'], errors='coerce').fillna(0).astype(int)
df_streets = df_warszawa[df_warszawa['UlicaID'] != 0].copy()

def get_last_location_part(location_str):
    if not isinstance(location_str, str): return np.nan
    last_part = location_str.split(',')[-1].strip().lower()
    return re.sub(r'^(ul\.|al\.|pl\.)\s*', '', last_part)

df_streets['Nazwa_Wydedukowana'] = df_streets['LocationText'].apply(get_last_location_part)
df_streets.dropna(subset=['Nazwa_Wydedukowana'], inplace=True)
df_deduced = df_streets[['UlicaID', 'Nazwa_Wydedukowana']].drop_duplicates(subset='UlicaID')

# ZMIANA: Nadajemy kolumnie z ID nazwę 'LocationID', aby pasowała do drugiej ramki
df_deduced.rename(columns={'UlicaID': 'LocationID'}, inplace=True)

# --- Krok 3: Przygotowanie oficjalnego słownika ---
df_official = df_meta_raw.copy()
df_official.columns = ['LocationId', 'ParentId', 'Name', 'Type', 'Path']
df_official.rename(columns={'LocationId': 'LocationID', 'Name': 'Nazwa_Oficjalna'}, inplace=True)
df_official_subset = df_official[['LocationID', 'Nazwa_Oficjalna']]

# --- Krok 4: Łączenie i weryfikacja ---
print("\nŁączenie i porównywanie słowników...")
df_merged = pd.merge(df_deduced, df_official_subset, on='LocationID', how='left')
df_merged['Nazwa_Oficjalna'].fillna('Brak w bazie oficjalnej', inplace=True)

def choose_best_name(row):
    official_name = str(row['Nazwa_Oficjalna']).lower()
    if official_name != 'brak w bazie oficjalnej' and official_name != 'nan':
        return row['Nazwa_Oficjalna']
    return row['Nazwa_Wydedukowana']

df_merged['Nazwa_Finalna'] = df_merged.apply(choose_best_name, axis=1)

# --- Krok 5: Zapis i wyświetlenie finalnego słownika ---
final_dictionary_df = df_merged[['LocationID', 'Nazwa_Finalna']].copy()
final_dictionary_df.rename(columns={'Nazwa_Finalna': 'Nazwa'}, inplace=True)

print(f"\nZbudowano finalny, zweryfikowany słownik z {len(final_dictionary_df)} wpisów.")

output_filename = 'slownik_finalny_zweryfikowany.csv'
print(f"Zapisywanie słownika do pliku: {output_filename}...")
final_dictionary_df.to_csv(output_filename, index=False, sep=';')
print("Zapis zakończony sukcesem!")

print("\nPróbka 20 losowych wierszy z finalnego, zweryfikowanego słownika:")
display(final_dictionary_df.sample(20, random_state=42))

--- Krok 1: Wczytanie danych ---
Plik 'saleflats_mazowieckie_c.csv' wczytany.


C:\Users\tmirk\AppData\Local\Temp\ipykernel_22712\3847162788.py:14: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta_raw = pd.read_csv('lokalizacja.csv', sep=',', header=None, on_bad_lines='skip')


Plik 'lokalizacja.csv' wczytany.

Łączenie i porównywanie słowników...

Zbudowano finalny, zweryfikowany słownik z 2708 wpisów.
Zapisywanie słownika do pliku: slownik_finalny_zweryfikowany.csv...
Zapis zakończony sukcesem!

Próbka 20 losowych wierszy z finalnego, zweryfikowanego słownika:


,LocationID,Nazwa
1044,515548,"Kompanii ak \goplana"""
439,361128,Szegedyńska
1729,514352,Pielęgniarek
296,414096,Kwitnąca
2211,480945,Tawerny
479,364478,Belgijska
1084,376107,Lutocińska
2080,500370,Cicha dolina
2204,305815,Leśniczówka
318,295727,Dymińska
